전통적 통계 방식들을 테스트 하는 코드들이였습니다. 몇개 지워져 있어 제대로 동작하지 않을 수 있습니다.

In [55]:
import numpy as np
from common.functions import sigmoid

class LogisticRegression:
    def __init__(self, lr=0.01, num_iter=1000):
        self.lr = lr
        self.num_iter = num_iter
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        m, n = X.shape
        self.weights = np.random.rand(n) #가중치 초기화: 랜덤 설정
        self.bias = np.random.rand(1) #편향 초기화: 랜덤 설정
        for i in range(self.num_iter):
            linear_model = np.dot(X, self.weights) + self.bias
            y_pred = sigmoid(linear_model)
            self.weights += self.lr * ((1 / m) * np.dot(X.T, (y_pred - y))) *-1
            self.bias += self.lr * ((1 / m) * np.sum(y_pred - y))*-1

    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_pred = sigmoid(linear_model)
        return [1 if i > 0.5 else 0 for i in y_pred]

In [56]:
class SVM:
    def __init__(self, lr=0.001, lambda_param=0.01, num_iter=1000):
        self.lr = lr
        self.lambda_param = lambda_param
        self.num_iter = num_iter
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        y_temp = np.where(y <= 0, -1, 1)
        m, n = X.shape
        self.weights = np.random.rand(n) #가중치 초기화: 랜덤 설정
        self.bias = np.random.rand(1) #편향 초기화: 랜덤 설정

        for _ in range(self.num_iter):
            for idx, x_i in enumerate(X):
                condition = y_temp[idx] * (np.dot(x_i, self.weights) - self.bias) >= 1
                if condition:
                    self.weights -= self.lr * (2 * self.lambda_param * self.weights)
                else:
                    self.weights -= self.lr * (2 * self.lambda_param * self.weights - np.dot(x_i, y_temp[idx]))
                    self.bias -= self.lr * y_temp[idx]

    def predict(self, X):
        linear_model = np.dot(X, self.weights) - self.bias
        return np.sign(linear_model)

In [57]:
class LinearDiscriminantAnalysis:
    def __init__(self):
        self.means = None
        self.priors = None
        self.covariance = None

    def fit(self, X, y):
        self.means = {}
        self.priors = {}
        classes = np.unique(y)
        n = X.shape[1]
        self.covariance = np.zeros((n, n))

        for cls in classes:
            X_c = X[y == cls]
            self.means[cls] = np.mean(X_c, axis=0)
            self.priors[cls] = X_c.shape[0] / X.shape[0]
            self.covariance += np.cov(X_c, rowvar=False) * (X_c.shape[0] - 1)

        self.covariance /= (X.shape[0] - len(classes))
        # 작은 값 추가하여 행렬이 비특이행렬이 되도록 처리
        self.covariance += np.eye(n) * 1e-6

    def predict(self, X):
        inv_cov = np.linalg.inv(self.covariance)
        scores = []
        for x in X:
            class_scores = {}
            for cls in self.means:
                mean_vec = self.means[cls]
                score = x @ inv_cov @ mean_vec - 0.5 * mean_vec.T @ inv_cov @ mean_vec + np.log(self.priors[cls])
                class_scores[cls] = score
            scores.append(max(class_scores, key=class_scores.get))
        return np.array(scores)


In [58]:
class GaussianNaiveBayes:
    def __init__(self):
        self.classes = None
        self.means = None
        self.variances = None
        self.priors = None

    def fit(self, X, y):
        self.classes = np.unique(y)
        self.means = {}
        self.variances = {}
        self.priors = {}

        for cls in self.classes:
            X_c = X[y == cls]
            self.means[cls] = np.mean(X_c, axis=0)
            self.variances[cls] = np.var(X_c, axis=0)
            self.priors[cls] = X_c.shape[0] / X.shape[0]

    def predict(self, X):
        posteriors = []

        for x in X:
            posterior = []
            for cls in self.classes:
                prior = np.log(self.priors[cls])
                class_conditional = -0.5 * np.sum(np.log(2. * np.pi * self.variances[cls])) - 0.5 * np.sum(((x - self.means[cls]) ** 2) / (self.variances[cls]))
                posterior.append(prior + class_conditional)
            posteriors.append(self.classes[np.argmax(posterior)])

        return np.array(posteriors)